# CHAPTER 09. 추천 시스템

## 05. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
" 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천하는 것이다. 이렇게 영화(또는 상품/서비스) 간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠(장르, 감동, 배우, 평점 등)를 기반으로 하는 방식이 바로 콘텐츠 기반 필터링이다.
- 장르 칼럼 값의 유사도를 비교한 뒤 그중 높은 평점을 가지는 영화를 추천하는 방식이다.

### 데이터 로딩 및 가공

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('/Users/wizdom/Desktop/data_analysis/파이썬 머신러닝 완벽가이드/실습 데이터/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


4803개의 레코드와 20개의 피처로 구성. 영화 제목, 예산, 장르, 등 영화에 대한 다양한 메타 정보를 가짐

In [3]:
# 콘텐츠기반 필터링 추천 분석에 사용할 주요 칼럼만 추출해 dataframe 만들기

movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count',
                    'popularity', 'keywords', 'overview']]

In [4]:
# 리스트 내부에 여러개의 딕셔너리가 있는 형태의 문자열인 두 칼럼 확인

pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres 칼럼은 여러 개의 개별 장르 데이터를 가지고 있고, 개별 장르의 명칭은 딕셔너리의 키인 'name'으로 추출할 수 있다. keywords 칼럼 역시 마찬가지 구조를 가지고 잇따.

In [5]:
# 두 칼럼의 문자열 분해하여 개별 장르를 리스트 객체로 추출

from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

이제 두 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다.
- **ast 모듈의 literal_eval()** : 문자열이 의미하는 list[dict1, dict2]객체로 만든다.

In [10]:
print(movies_df[['genres']])

                                                                                                   genres
0     [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
1          [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
2            [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]
3     [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...
4     [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...
...                                                                                                   ...
4798          [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]
4799                                     [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800  [{'id': 35, 'name': 'Comedy'}, {'id': 18

In [ ]:
# 장르명만 리스트 객체로 추출